In [121]:
import numpy as np
import pandas as pd

In [122]:
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex','capital-gain','capital-loss', 'hours-per-week', 'native-country', 'income']
train_df = pd.read_csv('./data/adult.data', names=headers, skipinitialspace=True)

test_df = pd.read_csv('./data/adult.test', names=headers, skipinitialspace=True)

In [123]:
# replace ? with NAN
train_df[train_df == '?'] = np.nan
test_df[test_df == '?'] = np.nan

X_train = train_df.drop("income", axis=1)
Y_train = train_df["income"]
X_test = test_df.drop("income", axis=1)
Y_test = test_df["income"]
#occupation 1843
#workclass 1836
#native-country 583

In [124]:
from sklearn.impute import KNNImputer
# preprocessing

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

def preprocess(train_data, test_data):
    
    # categorical to numerical 
    categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
    oe = [None] * len(categorical)
    for i, feature in enumerate(categorical):
            oe[i] = LabelEncoder()
            train_data[feature] = oe[i].fit_transform(train_data[feature])
            test_data[feature] = oe[i].transform(test_data[feature])
        
        # impute missing value    
    imputer = KNNImputer(missing_values=np.nan, add_indicator=True)
    train_data = imputer.fit_transform(train_data)
    test_data = imputer.transform(test_data)

    for i, feature in enumerate(categorical):
            oe[i] = LabelEncoder()
            train_data[feature] = oe[i].inverse_transform(train_data[feature])
            test_data[feature] = oe[i].inverse_transform(test_data[feature])
    # standard scaling
    # scaler = StandardScaler()
    # train_data = pd.DataFrame(scaler.fit_transform(train_data), columns = X_train.columns)
    # test_data = pd.DataFrame(scaler.transform(test_data), columns = X_train.columns)
    
    return train_data, test_data

In [125]:
X_train,X_test = preprocess(X_train, X_test)
X_train.head(5)
# from sklearn.model_selection import train_test_split
# 
# validation_size = 0.20
# 
# X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,
#     test_size=validation_size,random_state=42)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices